# Integrated Labs Assignment: Simple Blockchain Implementation

This block will have the imported libraries. I will use hashlib and datetime, both for hashing of the blockchain

In [1]:
import hashlib
import datetime

This block is responsible for creating a single block itself, it first has the initialising method, taking 3 arguments initially, the hash for the previous block, the data the block has, and the timestamp. For the proof of concept we added nonce and difficulty. I used a static method to generate the genesis block, the reason it's a static method is because I will only call it once to create the genesis block, after that it's no longer needed.

In [2]:
class Block:
    def __init__(self, previous_block_hash, data, timestamp, difficulty=4):
        self.previous_block_hash = previous_block_hash
        self.data = data
        self.timestamp = timestamp
        self.nonce = 0
        self.difficulty = difficulty
        self.hash = self.proof_of_work()

    #This method creates the genesis block, the first block in the block chain.
    #I used static method because we'll only need to call it once
    @staticmethod
    def create_genesis_block():
        return Block("0", "Genesis Block", datetime.datetime.now())

    #Method that will handle hashing the blocks
    def get_hash(self):
        #each variable of a block is joined together as a string
        header_bin = (str(self.previous_block_hash) + str(self.data) + 
                      str(self.timestamp) + str(self.nonce)).encode()
        return hashlib.sha256(header_bin).hexdigest() #then this is hashed in SHA-256 format

    #Proof of work handling, this ensures the hash for every block starts with the set difficulty
    def proof_of_work(self):
        target = "0" * self.difficulty #For this simple example I picked 0 for the set difficulty, any block whoose hash doesn't start with 4 zero's is considered invalid
        while True:
            self.hash = self.get_hash() #Takes the actual hash
            if self.hash.startswith(target): #checks if the hash starts with said target (0 in this case)
                break # If it's found, it stops the loop
            self.nonce += 1 #increments nonce for a new hash
        return self.hash #returns the hash

This block is purely for generating the genesis block it calls the method in the Block class and creates the genesis block

In [3]:
block_chain = [Block.create_genesis_block()]

print("The Genesis Block has been created")
print("Hash: %s" % block_chain[-1].hash)

The Genesis Block has been created
Hash: 0000e16b52154bf3361c4f0d5fdec245970fdeb74f964c35b90e82c88dd4f1a5


This block is for bulk adding however many blocks we want.

In [4]:
num_blocks_to_add = 5 #Variable that represents hwoever many blocks we wish to add, I want to add 5

#For loop that will handle bulk adding
for i in range(1, num_blocks_to_add+1):
    block_chain.append(Block(block_chain[-1].hash, "DATA!", datetime.datetime.now())) #each iteration add a block

    #Simple bug fixing print statements
    print("Block #%d has been created." % i)
    print("Block #%d hash: %s" % (i, block_chain[i].hash))

Block #1 has been created.
Block #1 hash: 00004e907f16a62fcb8f6d448f034c35bb2f96957058e3a5a6bbfd24b912c248
Block #2 has been created.
Block #2 hash: 0000d481660f6779d8a30c715c15e5e30a8753e1843a97bc19823c8bab26fe99
Block #3 has been created.
Block #3 hash: 00002b8af12e6baa84f593b7ebf40f72f90419e938343de24d690baa5ed69697
Block #4 has been created.
Block #4 hash: 0000112be2ba72976c6726599cf5dd1c9a55f21ea87414941224a262becb24ca
Block #5 has been created.
Block #5 hash: 00002f5001c4f1880fb60d7aa41aea0b5a6cfdbecf8301542714638c8806cca0


This block holds the class for blockchain, this code's main function is the validate_chain method whoose purpose is to chech through the whole list to validate it's integrity.

In [5]:
class Blockchain:
    def __init__(self, chain=None, difficulty=4):
        self.difficulty = difficulty #Setting the difficulty for the proof of work
        if chain is None or len(chain) == 0: #If chain is empty create one with a genesis block
            self.chain = [Block.create_genesis_block()]
        else:
            self.chain = chain

    def add_block(self, data): #Method ot add a single block
        last_block = self.chain[-1] #select the last block of the chain
        new_block = Block(last_block.hash, data, datetime.datetime.now(), self.difficulty) #Create the new block
        self.chain.append(new_block) #Append the new block connecting it to the chain

    #Validating method
    def validate_chain(self):
        #For loop who will go through every block in the chain
        for i in range(1, len(self.chain)):
            current_block = self.chain[i] #Select current i block
            previous_block = self.chain[i - 1] #Take the previous block afterwards

            #Chck if the current block's hash is the same as the calculated hash to ensure it hasn't been manipulated
            if current_block.hash != current_block.get_hash():
                print(f"Block {i} has been tampered with!")
                return False

            #Same logic with the previous block
            if current_block.previous_block_hash != previous_block.hash:
                print(f"Block {i} has an invalid previous block hash!")
                return False

            #Check if the hash starts with 4 zero's
            if not current_block.hash.startswith("0" * self.difficulty):
                print(f"Block {i} does not satisfy Proof of Work!")
                return False

            #Simple bug fixing printing
            print(f"Block {i}")

        return True

Simple block to test the validation method

In [6]:
my_blockchain = Blockchain(chain=block_chain) #Set the blocks I've created into the blockchain clas

is_valid = my_blockchain.validate_chain() #Call the validation method

#Simple bug fixing prints
if is_valid:
    print("The blockchain is valid!")
else:
    print("The blockchain is invalid!")

Block 1
Block 2
Block 3
Block 4
Block 5
The blockchain is valid!
